# 🌍 NYC Environmental & Traffic Correlation Analysis
## Blade Runner Surveillance System - Weather × Air Quality × Traffic

This notebook correlates NYC traffic data with environmental conditions including:
- **Weather Data** (NOAA weather stations)
- **Air Quality** (EPA AQI monitoring)
- **Traffic Events** (511NY incidents/construction)
- **Traffic Speeds** (Real-time congestion)

*"The climate of a city defines its character... and its traffic patterns."*

In [ ]:
-- Set context
USE DATABASE DEMO;
USE SCHEMA DEMO;
USE WAREHOUSE INGEST;

---
## 📊 Data Sources Overview

In [ ]:
-- Data volume summary across all environmental and traffic tables
SELECT 'NOAAWEATHER' as source, COUNT(*) as records, 'Weather' as category FROM NOAAWEATHER
UNION ALL SELECT 'AQ', COUNT(*), 'Air Quality' FROM AQ
UNION ALL SELECT 'WEATHER_OBSERVATIONS', COUNT(*), 'Weather' FROM WEATHER_OBSERVATIONS
UNION ALL SELECT 'NYC_CAMERA_DATA', COUNT(*), 'Traffic' FROM NYC_CAMERA_DATA
UNION ALL SELECT 'NYC_TRAFFIC_EVENTS', COUNT(*), 'Traffic' FROM NYC_TRAFFIC_EVENTS
UNION ALL SELECT 'NYC_TRAFFIC_SPEEDS', COUNT(*), 'Traffic' FROM NYC_TRAFFIC_SPEEDS
ORDER BY category, source;

---
## 🌤️ Weather Data Analysis (NOAA)

In [ ]:
-- Current weather conditions by station
SELECT 
    station_id,
    location,
    weather,
    temp_f,
    relative_humidity,
    wind_mph,
    wind_dir,
    visibility_mi,
    pressure_in,
    observation_time
FROM NOAAWEATHER
WHERE observation_time IS NOT NULL
QUALIFY ROW_NUMBER() OVER (PARTITION BY station_id ORDER BY updated_at DESC) = 1
ORDER BY station_id
LIMIT 20;

In [ ]:
-- Weather conditions distribution
SELECT 
    weather as condition,
    COUNT(*) as observation_count,
    ROUND(AVG(temp_f), 1) as avg_temp_f,
    ROUND(AVG(wind_mph), 1) as avg_wind_mph,
    ROUND(AVG(visibility_mi), 1) as avg_visibility_mi
FROM NOAAWEATHER
WHERE weather IS NOT NULL AND weather != ''
GROUP BY weather
ORDER BY observation_count DESC
LIMIT 15;

In [ ]:
-- Temperature distribution across stations
SELECT 
    location,
    ROUND(MIN(temp_f), 1) as min_temp,
    ROUND(AVG(temp_f), 1) as avg_temp,
    ROUND(MAX(temp_f), 1) as max_temp,
    ROUND(AVG(relative_humidity), 1) as avg_humidity,
    COUNT(*) as observations
FROM NOAAWEATHER
WHERE temp_f IS NOT NULL
GROUP BY location
ORDER BY avg_temp DESC
LIMIT 20;

---
## 🌬️ Air Quality Analysis (EPA AQI)

In [ ]:
-- Air Quality Index overview by reporting area
SELECT 
    reportingarea,
    statecode,
    parametername,
    ROUND(AVG(aqi), 1) as avg_aqi,
    MAX(aqi) as max_aqi,
    MIN(aqi) as min_aqi,
    COUNT(*) as readings,
    MODE(categoryname) as most_common_category
FROM AQ
GROUP BY reportingarea, statecode, parametername
ORDER BY avg_aqi DESC
LIMIT 20;

In [ ]:
-- AQI category distribution
SELECT 
    categoryname,
    categorynumber,
    COUNT(*) as reading_count,
    ROUND(AVG(aqi), 1) as avg_aqi,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM AQ
GROUP BY categoryname, categorynumber
ORDER BY categorynumber;

In [ ]:
-- Pollutant type analysis
SELECT 
    parametername as pollutant,
    COUNT(*) as readings,
    ROUND(AVG(aqi), 1) as avg_aqi,
    MAX(aqi) as max_aqi,
    COUNT(DISTINCT reportingarea) as monitoring_areas
FROM AQ
GROUP BY parametername
ORDER BY avg_aqi DESC;

In [ ]:
-- Hourly AQI trends
SELECT 
    hourobserved,
    ROUND(AVG(aqi), 1) as avg_aqi,
    COUNT(*) as readings,
    MAX(aqi) as peak_aqi
FROM AQ
WHERE hourobserved IS NOT NULL
GROUP BY hourobserved
ORDER BY hourobserved;

---
## 🔗 Weather × Traffic Correlation Analysis

In [ ]:
-- Weather conditions impact on visibility (proxy for driving conditions)
SELECT 
    CASE 
        WHEN visibility_mi >= 10 THEN 'Excellent (10+ mi)'
        WHEN visibility_mi >= 5 THEN 'Good (5-10 mi)'
        WHEN visibility_mi >= 2 THEN 'Moderate (2-5 mi)'
        WHEN visibility_mi >= 1 THEN 'Poor (1-2 mi)'
        ELSE 'Very Poor (<1 mi)'
    END as visibility_category,
    weather,
    COUNT(*) as observations,
    ROUND(AVG(temp_f), 1) as avg_temp,
    ROUND(AVG(wind_mph), 1) as avg_wind
FROM NOAAWEATHER
WHERE visibility_mi IS NOT NULL
GROUP BY visibility_category, weather
ORDER BY visibility_category, observations DESC
LIMIT 30;

In [ ]:
-- Wind conditions analysis (impacts traffic safety)
SELECT 
    CASE 
        WHEN wind_mph >= 30 THEN '🌪️ High Wind (30+ mph)'
        WHEN wind_mph >= 20 THEN '💨 Strong (20-30 mph)'
        WHEN wind_mph >= 10 THEN '🍃 Moderate (10-20 mph)'
        WHEN wind_mph >= 5 THEN '🌿 Light (5-10 mph)'
        ELSE '😌 Calm (<5 mph)'
    END as wind_category,
    COUNT(*) as observations,
    ROUND(AVG(temp_f), 1) as avg_temp,
    ROUND(AVG(visibility_mi), 1) as avg_visibility,
    LISTAGG(DISTINCT wind_dir, ', ') WITHIN GROUP (ORDER BY wind_dir) as wind_directions
FROM NOAAWEATHER
WHERE wind_mph IS NOT NULL
GROUP BY wind_category
ORDER BY wind_category;

---
## 🚨 Air Quality × Traffic Events Correlation

In [ ]:
-- Air quality health advisory levels
SELECT 
    categoryname as aqi_category,
    CASE 
        WHEN categorynumber = '1' THEN '✅ No health impacts expected'
        WHEN categorynumber = '2' THEN '⚠️ Unusually sensitive people may be affected'
        WHEN categorynumber = '3' THEN '🟡 Sensitive groups may experience health effects'
        WHEN categorynumber = '4' THEN '🟠 Everyone may begin to experience effects'
        WHEN categorynumber = '5' THEN '🔴 Health alert: serious effects for everyone'
        ELSE '🟣 Health emergency'
    END as health_advisory,
    COUNT(*) as occurrences,
    ROUND(AVG(aqi), 0) as typical_aqi
FROM AQ
GROUP BY categoryname, categorynumber
ORDER BY categorynumber;

In [ ]:
-- Geographic coverage: monitoring stations with coordinates
SELECT 
    'Weather Stations' as data_source,
    COUNT(DISTINCT station_id) as stations,
    ROUND(AVG(latitude), 4) as center_lat,
    ROUND(AVG(longitude), 4) as center_lon
FROM NOAAWEATHER
WHERE latitude IS NOT NULL
UNION ALL
SELECT 
    'AQ Monitoring Areas',
    COUNT(DISTINCT reportingarea),
    ROUND(AVG(latitude), 4),
    ROUND(AVG(longitude), 4)
FROM AQ
WHERE latitude IS NOT NULL;

---
## 📈 Environmental Conditions Dashboard Summary

In [ ]:
-- Executive Summary: Current Environmental Status
SELECT 
    '🌡️ Temperature Range' as metric,
    CONCAT(ROUND(MIN(temp_f), 0), '°F - ', ROUND(MAX(temp_f), 0), '°F') as current_value,
    'NOAA Weather' as source
FROM NOAAWEATHER WHERE temp_f IS NOT NULL
UNION ALL
SELECT 
    '💨 Wind Speed Range',
    CONCAT(ROUND(MIN(wind_mph), 0), ' - ', ROUND(MAX(wind_mph), 0), ' mph'),
    'NOAA Weather'
FROM NOAAWEATHER WHERE wind_mph IS NOT NULL
UNION ALL
SELECT 
    '👁️ Visibility Range',
    CONCAT(ROUND(MIN(visibility_mi), 1), ' - ', ROUND(MAX(visibility_mi), 1), ' miles'),
    'NOAA Weather'
FROM NOAAWEATHER WHERE visibility_mi IS NOT NULL
UNION ALL
SELECT 
    '🌬️ AQI Range',
    CONCAT(MIN(aqi), ' - ', MAX(aqi), ' (', MODE(categoryname), ')'),
    'EPA AirNow'
FROM AQ WHERE aqi IS NOT NULL
UNION ALL
SELECT 
    '📍 Monitoring Coverage',
    CONCAT(COUNT(DISTINCT station_id), ' weather stations'),
    'NOAA Weather'
FROM NOAAWEATHER
UNION ALL
SELECT 
    '📍 AQ Monitoring',
    CONCAT(COUNT(DISTINCT reportingarea), ' reporting areas'),
    'EPA AirNow'
FROM AQ;

---
## 📍 Geospatial Analysis

Connect weather and air quality data using latitude/longitude coordinates with Snowflake geospatial functions.

In [ ]:
-- Weather stations with geospatial points (nearest to NYC)
SELECT 
    station_id,
    location,
    weather AS conditions,
    temp_f,
    wind_mph,
    visibility_mi,
    latitude,
    longitude,
    ST_MAKEPOINT(longitude, latitude) AS geo_point,
    ROUND(ST_DISTANCE(
        ST_MAKEPOINT(longitude, latitude), 
        ST_MAKEPOINT(-74.0060, 40.7128)  -- NYC coordinates
    ) / 1609.34, 1) AS distance_from_nyc_miles
FROM NOAAWEATHER
WHERE latitude IS NOT NULL AND longitude IS NOT NULL
ORDER BY distance_from_nyc_miles
LIMIT 10;

In [ ]:
-- Air quality monitors near major cities using ST_DWITHIN
SELECT 
    reportingarea,
    statecode,
    parametername AS pollutant,
    aqi,
    categoryname AS category,
    latitude,
    longitude,
    ROUND(ST_DISTANCE(
        ST_MAKEPOINT(longitude, latitude),
        ST_MAKEPOINT(-74.0060, 40.7128)
    ) / 1609.34, 1) AS distance_from_nyc_miles
FROM AQ
WHERE ST_DWITHIN(
    ST_MAKEPOINT(longitude, latitude),
    ST_MAKEPOINT(-74.0060, 40.7128),
    160934  -- 100 miles in meters
)
ORDER BY distance_from_nyc_miles;

In [ ]:
-- H3 hexagonal grid analysis for spatial aggregation
SELECT 
    H3_LATLNG_TO_CELL(latitude, longitude, 5) AS h3_cell_res5,
    COUNT(DISTINCT station_id) AS weather_station_count,
    ROUND(AVG(temp_f), 1) AS avg_temp_f,
    ROUND(AVG(wind_mph), 1) AS avg_wind_mph,
    COUNT(*) AS total_observations
FROM NOAAWEATHER
WHERE latitude IS NOT NULL AND longitude IS NOT NULL
GROUP BY h3_cell_res5
ORDER BY weather_station_count DESC
LIMIT 20;

In [ ]:
-- Join weather and air quality data by proximity using H3 cells
WITH weather_h3 AS (
    SELECT 
        H3_LATLNG_TO_CELL(latitude, longitude, 6) AS h3_cell,
        AVG(temp_f) AS avg_temp_f,
        AVG(wind_mph) AS avg_wind_mph,
        AVG(visibility_mi) AS avg_visibility
    FROM NOAAWEATHER
    WHERE latitude IS NOT NULL
    GROUP BY h3_cell
),
aq_h3 AS (
    SELECT 
        H3_LATLNG_TO_CELL(latitude, longitude, 6) AS h3_cell,
        AVG(aqi) AS avg_aqi,
        MAX(aqi) AS max_aqi,
        MODE(categoryname) AS typical_category
    FROM AQ
    WHERE latitude IS NOT NULL
    GROUP BY h3_cell
)
SELECT 
    COALESCE(w.h3_cell, a.h3_cell) AS h3_region,
    ROUND(w.avg_temp_f, 1) AS temperature_f,
    ROUND(w.avg_wind_mph, 1) AS wind_mph,
    ROUND(w.avg_visibility, 1) AS visibility_miles,
    ROUND(a.avg_aqi, 0) AS avg_aqi,
    a.max_aqi,
    a.typical_category AS air_quality_level
FROM weather_h3 w
FULL OUTER JOIN aq_h3 a ON w.h3_cell = a.h3_cell
WHERE w.avg_temp_f IS NOT NULL OR a.avg_aqi IS NOT NULL
ORDER BY avg_aqi DESC NULLS LAST
LIMIT 20;

In [ ]:
-- Find nearest weather station to each air quality monitor
SELECT 
    aq.reportingarea,
    aq.parametername AS pollutant,
    aq.aqi,
    aq.categoryname AS aq_category,
    w.station_id AS nearest_weather_station,
    w.location AS weather_location,
    w.weather AS conditions,
    w.temp_f,
    w.wind_mph,
    ROUND(ST_DISTANCE(
        ST_MAKEPOINT(aq.longitude, aq.latitude),
        ST_MAKEPOINT(w.longitude, w.latitude)
    ) / 1609.34, 1) AS distance_miles
FROM AQ aq
CROSS JOIN NOAAWEATHER w
WHERE aq.latitude IS NOT NULL AND w.latitude IS NOT NULL
QUALIFY ROW_NUMBER() OVER (
    PARTITION BY aq.reportingarea, aq.parametername 
    ORDER BY ST_DISTANCE(
        ST_MAKEPOINT(aq.longitude, aq.latitude),
        ST_MAKEPOINT(w.longitude, w.latitude)
    )
) = 1
ORDER BY aq.aqi DESC
LIMIT 15;

In [ ]:
-- HAVERSINE distance calculation for precise geographic distances
SELECT 
    a.reportingarea AS aq_location,
    a.aqi,
    w.location AS weather_station,
    w.temp_f,
    ROUND(HAVERSINE(a.latitude, a.longitude, w.latitude, w.longitude) / 1609.34, 2) AS haversine_distance_miles
FROM AQ a
CROSS JOIN NOAAWEATHER w
WHERE a.reportingarea = 'New York City Region'
  AND a.latitude IS NOT NULL AND w.latitude IS NOT NULL
QUALIFY ROW_NUMBER() OVER (ORDER BY HAVERSINE(a.latitude, a.longitude, w.latitude, w.longitude)) <= 5;

In [ ]:
-- Use the custom geospatial function for environmental analysis at any location
-- Example: Analyze conditions near Times Square, NYC
SELECT * FROM TABLE(DEMO.DEMO.GET_ENVIRONMENTAL_CONDITIONS_AT_LOCATION(
    40.7580::FLOAT,  -- Times Square latitude
    -73.9855::FLOAT, -- Times Square longitude
    50::FLOAT        -- 50 mile radius
));

In [ ]:
-- Find weather stations within 25 miles of JFK Airport
SELECT * FROM TABLE(DEMO.DEMO.GET_WEATHER_STATIONS_WITHIN_RADIUS(
    40.6413::FLOAT,  -- JFK latitude
    -73.7781::FLOAT, -- JFK longitude  
    25::FLOAT        -- 25 mile radius
));

---
## 🗺️ Geospatial Functions Reference

| Function | Description |
|----------|-------------|
| `ST_MAKEPOINT(lon, lat)` | Create GEOGRAPHY point from coordinates |
| `ST_DISTANCE(geo1, geo2)` | Distance in meters between two points |
| `ST_DWITHIN(geo1, geo2, meters)` | Check if points within distance |
| `HAVERSINE(lat1, lon1, lat2, lon2)` | Great circle distance in meters |
| `H3_LATLNG_TO_CELL(lat, lon, res)` | Convert to H3 hexagonal cell |
| `H3_CELL_TO_BOUNDARY(cell)` | Get cell polygon boundary |
| `H3_CELL_TO_POINT(cell)` | Get cell center point |

### Custom Functions Created
- `GET_NEAREST_WEATHER_STATION(lat, lon)` - Find closest weather station
- `GET_WEATHER_STATIONS_WITHIN_RADIUS(lat, lon, miles)` - Find all stations in radius
- `GET_NEAREST_AIR_QUALITY(lat, lon)` - Find closest AQI monitor  
- `GET_ENVIRONMENTAL_CONDITIONS_AT_LOCATION(lat, lon, radius)` - Full environmental report
- `CALC_DISTANCE_MILES(lat1, lon1, lat2, lon2)` - Distance in miles
- `ANALYZE_LOCATION_CONDITIONS(lat, lon, radius)` - Stored procedure for JSON report

---
## 📝 Summary

This notebook provides comprehensive environmental analysis including:

1. **Weather Monitoring**: NOAA weather station data with temperature, humidity, wind, and visibility
2. **Air Quality**: EPA AQI measurements with pollutant breakdowns and health advisories
3. **Geographic Coverage**: Station/monitoring area locations for spatial analysis
4. **Correlation Insights**: Weather conditions impact on traffic safety

### Key Tables Used
| Table | Source | Records | Description |
|-------|--------|---------|-------------|
| `NOAAWEATHER` | NOAA NWS | 6,325+ | Weather observations from NOAA stations |
| `AQ` | EPA AirNow | 1,392+ | Air Quality Index measurements |
| `WEATHER_OBSERVATIONS` | NOAA | 6,033+ | Additional weather data |
| `NYC_TRAFFIC_EVENTS` | 511NY | - | Traffic incidents/construction |
| `NYC_TRAFFIC_SPEEDS` | 511NY | - | Real-time speed data |

*"In the future, cities will breathe data... and we'll finally understand the patterns."*